In [2]:
!pip install transformers

     |████████████████████████████████| 4.9 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 13.8 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.9.1 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.
